In [1]:
import pathlib
import sys

In [2]:
import numpy as np
import joblib
import pandas as pd

In [3]:
root_dir_path = pathlib.Path("..")
data_dir_path = root_dir_path / "data"
processed_dir_path = data_dir_path / "processed"
test_path = processed_dir_path / "test_engineered.parquet"
models_dir_path = root_dir_path / "models"
model_path = models_dir_path / "model.joblib"
prediction_path = models_dir_path / "prediction.parquet"
src_dir_path = root_dir_path / "src"

In [4]:
sys.path.append(str(src_dir_path))

In [5]:
from constants import *

In [6]:
test = pd.read_parquet(test_path)

In [7]:
model = joblib.load(model_path)

In [8]:
dates = test["date"].unique()

for date in dates:
    is_date = test["date"] == date
    X = test.loc[is_date, features]
    y_pred = model.predict(X)
    y_pred = np.maximum(0.0, y_pred)
    test.loc[is_date, "demand"] = y_pred

    is_date = test["date"] == date + np.timedelta64(1, "D")

    if is_date.sum() > 0:
        test.loc[is_date, "demand_shift_1"] = y_pred

In [9]:
test.to_parquet(prediction_path)